# Sentiment Analysis usando Deep Learning para español en textos cortos

# 1. Data Cleaning 

We work on a dataset of tweets labelled as positive or negative, more information about the dataset 
can be found on http://www.sepln.org/workshops/tass/. 

# 2. Vocabulary

## 2.2 Getting train, validation and test subsets

In [1]:
import lib.xmlreader as xml
import lib.utils as ut
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import gensim.models.word2vec

In [2]:
train_docs = xml.readXML("../database/TASS/TASS2018/task1-Training.xml",[0,1,2,3])
val_docs   = xml.readXML("../database/TASS/TASS2018/task1-Development.xml",[0,1,2,3])

In [3]:
train_tweets = []
train_labels = []
for doc in train_docs:
    # train_tweets.append(ut.tokenize(doc.content, 0)['clean'])
    train_tweets.append(doc.content)
    train_labels.append(doc.polarity)

val_tweets = []
val_labels = []
for doc in val_docs:
    # test_tweets.append(ut.tokenize(doc.content, 0)['clean'])
    val_tweets.append(doc.content)
    val_labels.append(doc.polarity)

In [4]:
len(train_tweets), len(val_tweets)

(1000, 500)

In [8]:
POSI_train_docs = [train_docs[i] for i in range(len(train_labels)) if train_labels[i] == 0]
NEGA_train_docs = [train_docs[i] for i in range(len(train_labels)) if train_labels[i] == 1]
NEUT_train_docs = [train_docs[i] for i in range(len(train_labels)) if train_labels[i] == 2]
NONE_train_docs = [train_docs[i] for i in range(len(train_labels)) if train_labels[i] == 3]

level_train_docs = [POSI_train_docs,NEGA_train_docs,NEUT_train_docs,NONE_train_docs]

fmt = """Positive Sentences = {:d}
       \rNegative Sentences = {:d}
       \rNeutral  Sentences = {:d}
       \rNone Values        = {:d}"""

print(fmt.format(len(POSI_train_docs),
                 len(NEGA_train_docs),
                 len(NEUT_train_docs),
                 len(NONE_train_docs)))

Positive Sentences = 242
Negative Sentences = 231
Neutral  Sentences = 166
None Values        = 361


In [9]:
minSentLvl = min(len(POSI_train_docs),len(NEGA_train_docs),len(NEUT_train_docs),len(NONE_train_docs))

print('Minimum number of sentences per level : ', minSentLvl)

Minimum number of sentences per level :  166


In [11]:
import random

new_train_docs = []
for i in range(len(level_train_docs)):
    level_per = random.sample(level_train_docs[i],len(level_train_docs[i]))
    new_train_docs.append(level_per[:minSentLvl])

In [12]:
print("New size of sentences:\n")
fmt = """Positive Sentences = {:d}
       \rNegative Sentences = {:d}
       \rNeutral  Sentences = {:d}
       \rNone Values        = {:d}"""

print(fmt.format(len(new_train_docs[0]),
                 len(new_train_docs[1]),
                 len(new_train_docs[2]),
                 len(new_train_docs[3])))

New size of sentences:

Positive Sentences = 166
Negative Sentences = 166
Neutral  Sentences = 166
None Values        = 166


In [13]:
flat_train_docs = [item for sublist in new_train_docs for item in sublist]
shuf_train_docs = random.sample(flat_train_docs,len(flat_train_docs))

assert (len(shuf_train_docs) == 4 * minSentLvl)
print("shuf_train_docs size = ", len(shuf_train_docs))

shuf_train_docs size =  664


In [15]:
corpus = []
for doc in shuf_train_docs + val_docs:
    corpus.append(doc.content)

In [17]:
print("Sentences = ", (len(val_docs + shuf_train_docs)))

Sentences =  1164


In [18]:
shuf_train_labels = []
for doc in shuf_train_docs:
    shuf_train_labels.append(doc.polarity)
    
assert (len(shuf_train_labels) == len(shuf_train_docs))

# 3. Preparing Data

In [19]:
from gensim.models import KeyedVectors

def gensim_load_vec(path="../database/embeddings/SBW-vectors-300-min5.bin"):
    #use gensim_emb.wv.index2word if used this way to load vectors
    #gensim_emb = gensim.models.word2vec.Word2Vec.load(path)
    gensim_emb =  gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)
    vocab = gensim_emb.index2word
    vec = gensim_emb.syn0
    shape = gensim_emb.syn0.shape
    return gensim_emb, vec, shape, vocab

In [20]:
gensim_emb, vec, shape, vocab = gensim_load_vec()

/home/jose/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  
/home/jose/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  if __name__ == '__main__':


In [21]:
counter = CountVectorizer(tokenizer=ut.tokenizer)

In [23]:
X = counter.fit_transform(corpus)
print(X.shape)

(1164, 4640)


In [24]:
VOCAB_SIZE = X.shape[1]
embedding_matrix = np.zeros((VOCAB_SIZE, 300))
for word in vocab:
    try:
        i = counter.vocabulary_[word]
        embedding_matrix[i] = gensim_emb[word]
    except KeyError:
        pass
print(embedding_matrix.shape)

(4640, 300)


In [25]:
np.array_equal(embedding_matrix[counter.vocabulary_['hola']], gensim_emb['hola'])

True

In [47]:
test_docs   = xml.readXMLTest("../database/TASS/TASS2018/task1-Test.xml")

In [48]:
test_tweets = []
for doc in test_docs:
    test_tweets.append(doc.content)

In [56]:
assert (len(test_tweets) == 1428)

In [57]:
sequences = []
for tweet in corpus + test_tweets:
    sentence = []
    for word in ut.tokenizer(tweet):
        try:
            i = counter.vocabulary_[word]
            sentence.append(i)
        except KeyError:
            pass
        
    sequences.append(sentence)

In [59]:
assert (len(sequences) == (len(shuf_train_docs) + len(val_docs) + len(test_tweets)))

In [79]:
from keras.preprocessing.sequence import pad_sequences
x_train_seq = pad_sequences(sequences[:len(shuf_train_labels)], maxlen=45)
x_val_seq   = pad_sequences(sequences[len(shuf_train_labels):len(shuf_train_labels)+len(val_docs)], maxlen=45)
x_test_set  = pad_sequences(sequences[(len(shuf_train_labels)+len(val_docs)):], maxlen=45)
print('Shape of data train tensor:', x_train_seq.shape)
print('Shape of data val  tensor:', x_val_seq.shape)
print('Shape of data test  tensor:', x_test_set.shape)

Shape of data train tensor: (664, 45)
Shape of data val  tensor: (500, 45)
Shape of data test  tensor: (1428, 45)


# Model: Convolutional Neural Network

In [68]:
import tensorflow as tf
import keras 

from keras import backend as K
print(K.tensorflow_backend._get_available_gpus())


config = tf.ConfigProto(intra_op_parallelism_threads=4, \
                        inter_op_parallelism_threads=4, \
                        allow_soft_placement=True,\
                        device_count = {'CPU' : 1, 'GPU' : 0})

sess = tf.Session(config=config)
K.set_session(sess)
    
seed = 7

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model
from keras.layers import Conv1D, GlobalMaxPooling1D

tweet_input = Input(shape=(45,), dtype='int32')

tweet_encoder   = Embedding(VOCAB_SIZE, 300, weights=[embedding_matrix], input_length=45, trainable=False)(tweet_input)
bigram_branch   = Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1)(tweet_encoder)
bigram_branch   = GlobalMaxPooling1D()(bigram_branch)
trigram_branch  = Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1)(tweet_encoder)
trigram_branch  = GlobalMaxPooling1D()(trigram_branch)
fourgram_branch = Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu', strides=1)(tweet_encoder)
fourgram_branch = GlobalMaxPooling1D()(fourgram_branch)
merged = concatenate([bigram_branch, trigram_branch, fourgram_branch], axis=1)

merged = Dense(64, activation='relu')(merged)
merged = Dropout(0.5)(merged)

merged = Dense(4)(merged)
output = Activation('softmax')(merged)
model = Model(inputs=[tweet_input], outputs=[output])
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
model.summary()

[]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 45)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 45, 300)      1392000     input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 44, 100)      60100       embedding_3[0][0]                
__________________________________________________________________________________________________
conv1d_8 (Conv1D)               (None, 43, 100)      90100       embedding_3[0][0]                
__________________________________________________________________________________________________
conv1d_

In [69]:
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical

filepath="model/CNN_best_weights.{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

model.fit(x_train_seq, to_categorical(shuf_train_labels), batch_size=64, epochs=10,
                     validation_data=(x_test_seq, to_categorical(val_labels)), callbacks = [checkpoint])

Train on 664 samples, validate on 500 samples
Epoch 1/10
664/664 [==============================] - 1s 2ms/step - loss: 0.5773 - acc: 0.7481 - val_loss: 0.5558 - val_acc: 0.7500

Epoch 00001: val_acc improved from -inf to 0.75000, saving model to model/CNN_best_weights.01-0.7500.hdf5
Epoch 2/10
664/664 [==============================] - 1s 1ms/step - loss: 0.5429 - acc: 0.7508 - val_loss: 0.5904 - val_acc: 0.7500

Epoch 00002: val_acc did not improve from 0.75000
Epoch 3/10
664/664 [==============================] - 1s 1ms/step - loss: 0.5219 - acc: 0.7575 - val_loss: 0.5721 - val_acc: 0.7500

Epoch 00003: val_acc did not improve from 0.75000
Epoch 4/10
664/664 [==============================] - 1s 1ms/step - loss: 0.4894 - acc: 0.7684 - val_loss: 0.5223 - val_acc: 0.7510

Epoch 00004: val_acc improved from 0.75000 to 0.75100, saving model to model/CNN_best_weights.04-0.7510.hdf5
Epoch 5/10
664/664 [==============================] - 1s 1ms/step - loss: 0.4599 - acc: 0.7835 - val_loss: 

In [70]:
#from keras.models import load_model

#loaded_CNN_model = load_model('model/CNN_best_weights.05-0.5400.hdf5')
#loaded_CNN_model.evaluate(x=x_val_seq, y=y_validation)
model.evaluate(x=x_val_seq, y=to_categorical(val_labels))

500/500 [==============================] - 0s 461us/step


[0.5712553024291992, 0.744]

In [71]:
#from keras.models import load_model

#best_model = load_model('model/CNN_best_weights.01-0.7500.hdf5')

In [80]:
test_values = np.argmax(model.predict(x_test_set), axis = 1)

In [84]:
def getLabel(num):
    if num == 0:
        return 'N'
    elif num == 1:
        return 'P'
    elif num == 2:
        return 'NEU'
    elif num == 3:
        return 'NONE'

In [85]:
import xml.etree.ElementTree as ET

def putTestValue(xmlFIle, out):
    tree = ET.parse(xmlFIle)
    root = tree.getroot()

    tweets = []
    file = open(out,"w") 
    print(len(test_values))
    for i,tweet in enumerate(root.iter('tweet')): 
        #print(i)
        val = getLabel(test_values[i])
        ID = tweet.find('tweetid').text
        file.write(ID + "\t" + val + "\n")
    file.close() 

In [86]:
#test_values = np.argmax(best_model.predict(x_test_set), axis = 1)

In [87]:
putTestValue("../database/TASS/TASS2018/task1-Test.xml", "output_cnn_2018-1.txt")

1428
